# BoxOfficeMojo Data Cleaner
___
Best used with Collapsible Headers nbextension

**'''**

> ***'''***
>
> Load pandas dataframes pickled by boxOfficeMojoPipeline and perform cleaning operations
>
>1. Concatenates all years into one file, sorts by release date (most recent first)
>2. fills None with np.NaN
>3. adds profit % column
>4. adds ratings dummy columns 
>5. pickles file again under path defined at the very end
>
>
>
>_(TODO: transform ratings into 'youngest age allowed bins')._
>
> ***'''***

**'''**

In [1]:
import pandas as pd
import numpy as np

/Users/kibbles/anaconda3/envs/metis/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Load the dataframe from its pickle file and perform basic datatype transformations/cleaning

## Concatenate all years into one

In [2]:
df = pd.DataFrame()
years = range(2020,1979,-1)

for year in years:
    filepath = ('./data/mojo_'+str(year)+'_movies.pkl')
    year_df = pd.read_pickle(filepath)
    df = df.append(year_df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19449 entries, 0 to 66
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   index                 19449 non-null  int64         
 1   Movie_Title           19449 non-null  object        
 2   Domestic_Distributor  19017 non-null  object        
 3   Domestic_Total_Gross  19449 non-null  int64         
 4   Runtime               18735 non-null  object        
 5   Rating                10923 non-null  object        
 6   Release_Date          19449 non-null  datetime64[ns]
 7   Budget                4383 non-null   object        
 8   Cast1                 19127 non-null  object        
 9   Cast2                 18911 non-null  object        
 10  Cast3                 18758 non-null  object        
 11  Cast4                 18585 non-null  object        
 12  Director              19120 non-null  object        
 13  Writer             

## Add Intercept column

In [3]:
df['Intercept'] = 1

## Sort by release date, most recent first

In [4]:
df = df.drop_duplicates()
df = df.sort_values(['Release_Date'],ascending=False).reset_index()
df = df.drop(columns='level_0')
df = df.drop(columns='index')
df

,Movie_Title,Domestic_Distributor,Domestic_Total_Gross,Runtime,Rating,Release_Date,Budget,Cast1,Cast2,Cast3,Cast4,Director,Writer,Producer,Cinematographer,Intercept
0,Inside the Rain,Sky Island Films,8140,90,None,2020-03-13,None,Rosie Perez,Eric Roberts,Aaron Fisher,Ellen Toland,Aaron Fisher,Aaron Fisher,George LaVoo,Josh Fisher,1
1,The Hunt,Universal Pictures,5812500,90,R,2020-03-13,14000000,Betty Gilpin,Hilary Swank,Ike Barinholtz,Wayne Duvall,Craig Zobel,Nick Cuse,Jason Blum,Darran Tiernan,1
2,Never Rarely Sometimes Always,Focus Features,16565,101,PG-13,2020-03-13,None,Eliazar Jimenez,David Buneta,Christian Clements,Sam Dugger,Eliza Hittman,Eliza Hittman,Lia Buman,Hélène Louvart,1
3,Moondance,,2371,97,None,2020-03-13,None,Adam Conover,Cooper Flannigan,Sam Jones,Carolyn Rabbers,Cooper Flannigan,Cooper Flannigan,Chase Crawford,Greg Kraus,1
4,The Roads Not Taken,Bleecker Street Media,3518,85,R,2020-03-13,None,Javier Bardem,Elle Fanning,Salma Hayek,Branka Katic,Sally Potter,Sally Potter,Christopher Sheppard,Robbie Ryan,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15968,Modern Times,United Artists,163577,87,None,1936-02-05,None,Charles Chaplin,Paulette Goddard,Henry Bergman,Tiny Sandford,Charles Chaplin,Charles Chaplin,None,Ira H. Morgan,1
15969,M,Foremco Pictures Corporation,35566,117,None,1933-03-31,None,Peter Lorre,Ellen Widmann,Inge Landgut,Otto Wernicke,Fritz Lang,Thea von Harbou,None,Fritz Arno Wagner,1
15970,City Lights,United Artists,19181,87,G,1931-01-30,None,Charles Chaplin,Virginia Cherrill,Florence Lee,Harry Myers,Charles Chaplin,Charles Chaplin,None,Gordon Pollock,1
15971,Metropolis,Paramount Pictures,1236166,153,None,1927-05-06,None,Brigitte Helm,Alfred Abel,Gustav Fröhlich,Rudolf Klein-Rogge,Fritz Lang,Thea von Harbou,Erich Pommer,Karl Freund,1


## Fill None values with np.nan

In [5]:
df.fillna(value=np.nan, inplace=True)

## Add Profit % column

In [6]:
df['Profit_percent'] = (df['Domestic_Total_Gross'] - df['Budget']) / df['Budget']
df.head()

,Movie_Title,Domestic_Distributor,Domestic_Total_Gross,Runtime,Rating,Release_Date,Budget,Cast1,Cast2,Cast3,Cast4,Director,Writer,Producer,Cinematographer,Intercept,Profit_percent
0,Inside the Rain,Sky Island Films,8140,90.0,NaN,2020-03-13,NaN,Rosie Perez,Eric Roberts,Aaron Fisher,Ellen Toland,Aaron Fisher,Aaron Fisher,George LaVoo,Josh Fisher,1,NaN
1,The Hunt,Universal Pictures,5812500,90.0,R,2020-03-13,14000000.0,Betty Gilpin,Hilary Swank,Ike Barinholtz,Wayne Duvall,Craig Zobel,Nick Cuse,Jason Blum,Darran Tiernan,1,-0.584821
2,Never Rarely Sometimes Always,Focus Features,16565,101.0,PG-13,2020-03-13,NaN,Eliazar Jimenez,David Buneta,Christian Clements,Sam Dugger,Eliza Hittman,Eliza Hittman,Lia Buman,Hélène Louvart,1,NaN
3,Moondance,,2371,97.0,NaN,2020-03-13,NaN,Adam Conover,Cooper Flannigan,Sam Jones,Carolyn Rabbers,Cooper Flannigan,Cooper Flannigan,Chase Crawford,Greg Kraus,1,NaN
4,The Roads Not Taken,Bleecker Street Media,3518,85.0,R,2020-03-13,NaN,Javier Bardem,Elle Fanning,Salma Hayek,Branka Katic,Sally Potter,Sally Potter,Christopher Sheppard,Robbie Ryan,1,NaN


## Check the ratings

In [7]:
df['Rating'].value_counts()

R            4816
PG-13        2716
PG           1274
G              93
Not Rated      35
NC-17          22
Unrated         4
M/PG            2
TV-14           1
TV-PG           1
Name: Rating, dtype: int64

There are a few ratings that don't match - like Not Rated, Unrated, M/PG, TV-14, and TV-PG.

Let's convert these to their MPAA counterparts and then check the value counts again

In [8]:
rating_conversion = {'Unrated': 'Not Rated',
                     'M/PG': 'PG',
                     'TV-14':'PG-13',
                     'TV-PG':'PG'}
for rating in rating_conversion:
    df = df.replace(to_replace=rating, value=rating_conversion[rating])
df['Rating'].value_counts()

R            4816
PG-13        2717
PG           1277
G              93
Not Rated      39
NC-17          22
Name: Rating, dtype: int64

Now we can see that the extra ratings have been removed and folded into their MPAA counterparts

## Add ratings dummy columns

In [9]:
# Keep all dummy columns because so many films have no rating
ratings_dummies = pd.get_dummies(df['Rating'],drop_first=False)
df = pd.concat([df, ratings_dummies], axis=1)
# df = df.drop(columns = 'Rating')
df.head()

,Movie_Title,Domestic_Distributor,Domestic_Total_Gross,Runtime,Rating,Release_Date,Budget,Cast1,Cast2,Cast3,...,Producer,Cinematographer,Intercept,Profit_percent,G,NC-17,Not Rated,PG,PG-13,R
0,Inside the Rain,Sky Island Films,8140,90.0,NaN,2020-03-13,NaN,Rosie Perez,Eric Roberts,Aaron Fisher,...,George LaVoo,Josh Fisher,1,NaN,0,0,0,0,0,0
1,The Hunt,Universal Pictures,5812500,90.0,R,2020-03-13,14000000.0,Betty Gilpin,Hilary Swank,Ike Barinholtz,...,Jason Blum,Darran Tiernan,1,-0.584821,0,0,0,0,0,1
2,Never Rarely Sometimes Always,Focus Features,16565,101.0,PG-13,2020-03-13,NaN,Eliazar Jimenez,David Buneta,Christian Clements,...,Lia Buman,Hélène Louvart,1,NaN,0,0,0,0,1,0
3,Moondance,,2371,97.0,NaN,2020-03-13,NaN,Adam Conover,Cooper Flannigan,Sam Jones,...,Chase Crawford,Greg Kraus,1,NaN,0,0,0,0,0,0
4,The Roads Not Taken,Bleecker Street Media,3518,85.0,R,2020-03-13,NaN,Javier Bardem,Elle Fanning,Salma Hayek,...,Christopher Sheppard,Robbie Ryan,1,NaN,0,0,0,0,0,1


That oughta do it for now

## Add Score Columns (Feature Engineering)

Each score is calculated according to mean gross of all films _before_ the film in question.

i.e. an aggregate measure of the previous box-office power of the director/distributor/cast1 member

In [10]:
categories = ['Director', 'Writer', 'Producer', 'Cinematographer', 'Domestic_Distributor', 'Cast1', 'Cast2', 'Cast3', 'Cast4']

In [11]:
for category in categories:
    score_col_name = category+'_Score'
    score_col_name

    df[score_col_name] = (df.sort_values(['Release_Date'])
                                .groupby([category])
                                .Domestic_Total_Gross
                                .apply(lambda x: x.expanding().median().shift())
                            )
    # take log of score
    df[score_col_name] = df[score_col_name].apply(lambda x: np.log(x))
    # replace NaN values with 0's
    #df[score_col_name] = df[score_col_name].fillna(0)

## Take log of DTG and Budget

In [12]:
df['Log_DTG'] = df['Domestic_Total_Gross'].apply(lambda x: np.log(x))

In [13]:
df['Log_Bud'] = df['Budget'].apply(lambda x: np.log(x))

# Final checks
---

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15973 entries, 0 to 15972
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Movie_Title                 15973 non-null  object        
 1   Domestic_Distributor        15623 non-null  object        
 2   Domestic_Total_Gross        15973 non-null  int64         
 3   Runtime                     15560 non-null  float64       
 4   Rating                      8964 non-null   object        
 5   Release_Date                15973 non-null  datetime64[ns]
 6   Budget                      3331 non-null   float64       
 7   Cast1                       15691 non-null  object        
 8   Cast2                       15536 non-null  object        
 9   Cast3                       15422 non-null  object        
 10  Cast4                       15300 non-null  object        
 11  Director                    15684 non-null  object    

Looks good to me! Let's take a look at the first few rows.

In [15]:
df.head()

,Movie_Title,Domestic_Distributor,Domestic_Total_Gross,Runtime,Rating,Release_Date,Budget,Cast1,Cast2,Cast3,...,Writer_Score,Producer_Score,Cinematographer_Score,Domestic_Distributor_Score,Cast1_Score,Cast2_Score,Cast3_Score,Cast4_Score,Log_DTG,Log_Bud
0,Inside the Rain,Sky Island Films,8140,90.0,NaN,2020-03-13,NaN,Rosie Perez,Eric Roberts,Aaron Fisher,...,NaN,8.737934,NaN,11.629045,11.063877,14.496685,NaN,NaN,9.004545,NaN
1,The Hunt,Universal Pictures,5812500,90.0,R,2020-03-13,14000000.0,Betty Gilpin,Hilary Swank,Ike Barinholtz,...,NaN,17.145732,NaN,17.263232,NaN,16.003221,17.915033,NaN,15.575521,16.454568
2,Never Rarely Sometimes Always,Focus Features,16565,101.0,PG-13,2020-03-13,NaN,Eliazar Jimenez,David Buneta,Christian Clements,...,12.444278,NaN,11.191300,16.093152,NaN,NaN,NaN,NaN,9.715047,NaN
3,Moondance,,2371,97.0,NaN,2020-03-13,NaN,Adam Conover,Cooper Flannigan,Sam Jones,...,NaN,NaN,NaN,9.879297,NaN,NaN,NaN,NaN,7.771067,NaN
4,The Roads Not Taken,Bleecker Street Media,3518,85.0,R,2020-03-13,NaN,Javier Bardem,Elle Fanning,Salma Hayek,...,13.689267,13.892649,12.623175,14.988554,14.622493,15.549776,13.479712,11.972948,8.165648,NaN


# Pickle the data once again
___

Now we can pickle our saved data for use later

In [16]:
path=('./data/mojo_all_years_movies_cleaned.pkl')

df.to_pickle(path=path)

In [17]:
import os
os.system('say -v Daniel your data is pickled!');